In [ ]:
import pandas as pd
import altair as alt

# -------------------------
# 1️⃣ Preparar datos agregados
# -------------------------
# Cantidad por comuna y tipo de cocina
df_comuna = df.groupby(['Comuna', 'asiatico']).agg(
    cantidad=('Nombre','count'),
    rating_promedio=('Rating','mean')
).reset_index()

# Ordenar comunas por cantidad total de asiáticos para destacar tendencia
orden_comunas = df_comuna[df_comuna['asiatico'] == 'Asiático'].sort_values(
    'cantidad', ascending=False)['Comuna'].tolist()

df_comuna['Comuna'] = pd.Categorical(df_comuna['Comuna'], categories=orden_comunas, ordered=True)

# -------------------------
# 2️⃣ Crear gráfico de barras apiladas
# -------------------------
chart = alt.Chart(df_comuna).mark_bar().encode(
    x=alt.X('Comuna:N', sort=None, title='Comuna'),
    y=alt.Y('cantidad:Q', title='Cantidad de restaurantes'),
    color=alt.Color('asiatico:N', scale=alt.Scale(range=['green','blue']), title='Tipo de cocina'),
    tooltip=[
        alt.Tooltip('Comuna:N'),
        alt.Tooltip('asiatico:N', title='Tipo de cocina'),
        alt.Tooltip('cantidad:Q', title='Cantidad de restaurantes'),
        alt.Tooltip('rating_promedio:Q', title='Rating promedio', format=".1f")
    ]
).properties(
    width=700,
    height=400,
    title="Distribución de restaurantes veganos Asiáticos vs No Asiáticos por comuna"
)

chart


alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis automáticamente según palabras clave
# -------------------------
emoji_keywords = {
    'asiática': '🍜',
    'japonesa': '🍣',
    'coreana': '🥢',
    'tailandesa': '🍲',
    'india': '🍛',
    'italiana': '🍕',
    'latina': '🌮',
    'francesa': '🥐',
    'mediterránea': '🥗'
}

def asignar_emoji(tipo):
    tipo_lower = str(tipo).lower()
    for palabra, emoji in emoji_keywords.items():
        if palabra in tipo_lower:
            return emoji
    return '🥗'  # default

df['emoji'] = df['Tipo local'].apply(asignar_emoji)

# -------------------------
# 4️⃣ Crear columna de posición vertical dentro de cada tipo de cocina
# -------------------------
df['pos'] = df.groupby('Tipo local').cumcount()

# -------------------------
# 5️⃣ Crear gráfico pictograma por tipo de cocina con rating en X
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=25).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),  # Posición horizontal según rating
    y=alt.Y('Tipo local:N', title='Tipo de cocina'),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Tipo local:N', title='Tipo de cocina'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,
    height=400,
    title="Restaurantes veganos por tipo de cocina: pictograma por rating"
)

chart


alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'otra': '🍴'
}

# Función para asignar múltiples emojis si hay varios orígenes
def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ''.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear gráfico pictograma por origen geográfico con rating en X
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=25).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('Origen geográfico:N', title='Origen geográfico'),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,
    height=400,
    title="Restaurantes por origen geográfico: pictograma por rating"
)

chart


alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'otra': '🍴'
}

# Función para asignar múltiples emojis si hay varios orígenes
def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    # Separar los emojis con un espacio para que no queden pegados
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear gráfico pictograma por origen geográfico con rating en X
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('Origen geográfico:N', title='Origen geográfico',
            axis=alt.Axis(labelPadding=20),  # separa etiquetas
            sort=alt.EncodingSortField(field="Rating", op="mean", order="descending")),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=900,   # más ancho
    height=600,  # más alto
    title="Restaurantes por origen geográfico: pictograma por rating"
)

chart


alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear columna de posición vertical dentro de cada origen
# -------------------------
df['y_pos'] = df.groupby('Origen geográfico').cumcount()

# -------------------------
# 5️⃣ Crear gráfico tipo pictograma apilado
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('y_pos:O', title=None, axis=None),  # posición vertical apilada
    row=alt.Row('Origen geográfico:N', title='Origen geográfico', header=alt.Header(labelFontSize=14, labelAngle=0)),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=900,
    height=50  # altura de cada fila individual
).configure_view(
    strokeWidth=0
).configure_header(
    labelFontSize=14,
    labelAngle=0
)

chart


alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'asiática': '🍜',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear columna de posición vertical dentro de cada origen
# -------------------------
df['y_pos'] = df.groupby('Origen geográfico').cumcount()

# -------------------------
# 5️⃣ Crear gráfico tipo pictograma apilado
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('y_pos:O', title=None, axis=None),  # posición vertical apilada
    row=alt.Row('Origen geográfico:N', title='Origen geográfico', header=alt.Header(labelFontSize=14, labelAngle=0)),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=900,
    height=50  # altura de cada fila individual
).configure_view(
    strokeWidth=0
).configure_header(
    labelFontSize=14,
    labelAngle=0
)

chart


alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'asiática': '🍜',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear columna de posición vertical dentro de cada origen
# -------------------------
df['y_pos'] = df.groupby('Origen geográfico').cumcount()

# -------------------------
# 5️⃣ Crear gráfico tipo pictograma apilado y compacto
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=20).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('y_pos:O', title=None, axis=None),  # posición vertical apilada
    row=alt.Row('Origen geográfico:N', title='Origen geográfico',
                header=alt.Header(labelFontSize=12, labelAngle=0, labelPadding=5)),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,    # más compacto horizontal
    height=35     # altura de cada fila más ajustada
).configure_view(
    strokeWidth=0
).configure_header(
    labelFontSize=12,
    labelAngle=0
)

chart
# --- Código para crear el gráfico ---
chart = alt.Chart(df).mark_text(fontSize=18).encode(
    x=alt.X('Rating:Q', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('origen_pos:O', title=None, axis=None),
    text='emoji:N',
    tooltip=[ ... ]
).properties(width=700, height=30)

chart  # Esto lo muestra (aunque aparezca en blanco en Colab)
!pip install vl-convert-python
import vl_convert as vlc
from google.colab import files

vlc.vegalite_to_png(chart, "grafico_restaurantes.png", scale_factor=2.0)
files.download("grafico_restaurantes.png")



alt.Chart(...)

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np
# import vl_convert as vlc # Not needed for saving as HTML
from google.colab import files # Needed for downloading the HTML file

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'asiática': '🍜',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear columna de posición vertical dentro de cada origen
# -------------------------
df['y_pos'] = df.groupby('Origen geográfico').cumcount()
df['origen_pos'] = df['Origen geográfico'] + " " + df['y_pos'].astype(str)

# -------------------------
# 5️⃣ Crear gráfico compacto y funcional
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=18).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('origen_pos:O', title=None, axis=None),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(width=700, height=30).configure_view(strokeWidth=0)

chart  # Mostrar gráfico en Colab (aunque pueda aparecer en blanco)

# -------------------------
# 6️⃣ Exportar a HTML
# -------------------------
chart.save('grafico_restaurantes.html')

print("Chart saved as grafico_restaurantes.html")

# Optional: Download the HTML file
files.download("grafico_restaurantes.html")

Chart saved as grafico_restaurantes.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import altair as alt
import re
import numpy as np
# import vl_convert as vlc # Not needed for saving as HTML
from google.colab import files # Needed for downloading the HTML file

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según origen geográfico
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'asiática': '🍜',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear gráfico tipo pictograma horizontal
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('Origen geográfico:N', title='Origen geográfico', sort=None),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,
    height=100 + 50 * df['Origen geográfico'].nunique()  # altura proporcional a la cantidad de orígenes
).configure_view(
    strokeWidth=0
)

chart  # Mostrar en Colab (puede verse en blanco, pero el HTML se generará bien)

# -------------------------
# 5️⃣ Exportar a HTML
# -------------------------
chart.save('grafico_restaurantes.html')

print("Chart saved as grafico_restaurantes.html")

# Optional: Download the HTML file
files.download("grafico_restaurantes.html")

Chart saved as grafico_restaurantes.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# -------------------------
# 0️⃣ Instalar librería para exportar PNG
# -------------------------
# !pip install --upgrade vl-convert-python # Not needed if only saving as HTML
# !pip install pillow # Not needed if not converting to JPG

# -------------------------
# 1️⃣ Importar librerías
# -------------------------
import pandas as pd
import altair as alt
import re
import numpy as np
# import vl_convert as vlc # Not needed if only saving as HTML
from google.colab import files
# from PIL import Image # Not needed if not converting to JPG

# -------------------------
# 2️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 3️⃣ Extraer ranking numérico
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 4️⃣ Asignar emojis según origen geográfico
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'asiática': '🍜',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 5️⃣ Crear gráfico reducido para mostrar en Colab
# -------------------------
# Keeping the preview chart as it might still render in Colab for some users
df_subset = df.groupby('Origen geográfico').head(5)  # solo 5 restaurantes por origen

chart_preview = alt.Chart(df_subset).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('Origen geográfico:N', title='Origen geográfico'),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,
    height=50 + 50 * df_subset['Origen geográfico'].nunique()
).configure_view(strokeWidth=0)

chart_preview  # Se muestra en Colab

# -------------------------
# 6️⃣ Crear gráfico completo para exportar
# -------------------------
chart_full = alt.Chart(df).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('Origen geográfico:N', title='Origen geográfico'),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,
    height=100 + 50 * df['Origen geográfico'].nunique()
).configure_view(strokeWidth=0)


# -------------------------
# 7️⃣ Guardar HTML y descargar
# -------------------------
chart_full.save("grafico_restaurantes.html")
files.download("grafico_restaurantes.html")

print("Chart saved as grafico_restaurantes.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Chart saved as grafico_restaurantes.html


In [ ]:
# -------------------------
# 0️⃣ Instalar librerías necesarias
# -------------------------
!pip install --upgrade vl-convert-python pillow

# -------------------------
# 1️⃣ Importar librerías
# -------------------------
import pandas as pd
import altair as alt
import re
import numpy as np
import vl_convert as vlc
from google.colab import files
from PIL import Image

# -------------------------
# 2️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 3️⃣ Extraer ranking numérico
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 4️⃣ Asignar emojis según origen geográfico
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'asiática': '🍜',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 5️⃣ Crear gráfico reducido para mostrar en Colab
# -------------------------
df_subset = df.groupby('Origen geográfico').head(5)  # solo 5 restaurantes por origen

chart_preview = alt.Chart(df_subset).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('Origen geográfico:N', title='Origen geográfico'),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,
    height=50 + 50 * df_subset['Origen geográfico'].nunique()
).configure_view(strokeWidth=0)

chart_preview  # Se muestra en Colab

# -------------------------
# 6️⃣ Crear gráfico completo para exportar
# -------------------------
chart_full = alt.Chart(df).mark_text(fontSize=30).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('Origen geográfico:N', title='Origen geográfico'),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,
    height=100 + 50 * df['Origen geográfico'].nunique()
).configure_view(strokeWidth=0)

# -------------------------
# 7️⃣ Guardar HTML
# -------------------------
chart_full.save("grafico_restaurantes.html")
files.download("grafico_restaurantes.html")

# -------------------------
# 8️⃣ Exportar PNG
# -------------------------
vlc.vegalite_to_png(chart_full, "grafico_restaurantes.png")
files.download("grafico_restaurantes.png")

# -------------------------
# 9️⃣ Convertir PNG a JPG
# -------------------------
im = Image.open("grafico_restaurantes.png")
im.convert("RGB").save("grafico_restaurantes.jpg", quality=95)
files.download("grafico_restaurantes.jpg")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

TypeError: vegalite_to_png() got an unexpected keyword argument 'scale_factor'

In [3]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'asiática': '🍜',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear columna de posición vertical dentro de cada origen
# -------------------------
df['y_pos'] = df.groupby('Origen geográfico').cumcount()

# -------------------------
# 5️⃣ Crear gráfico tipo pictograma apilado y compacto
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=20).encode(
    x=alt.X('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    y=alt.Y('y_pos:O', title=None, axis=None),  # posición vertical apilada
    row=alt.Row('Origen geográfico:N', title='Origen geográfico',
                header=alt.Header(labelFontSize=12, labelAngle=0, labelPadding=5)),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=700,    # más compacto horizontal
    height=35     # altura de cada fila más ajustada
).configure_view(
    strokeWidth=0
).configure_header(
    labelFontSize=12,
    labelAngle=0
)

chart

alt.Chart(...)

In [31]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Extraer ranking numérico (opcional)
# -------------------------
def extraer_ranking(texto):
    if pd.isna(texto):
        return np.nan
    match = re.search(r'\d+', str(texto).replace('.', ''))
    if match:
        return int(match.group())
    else:
        return np.nan

df['ranking_num'] = df['Ranking en Santiago'].apply(extraer_ranking)

# -------------------------
# 3️⃣ Asignar emojis según todos los orígenes presentes
# -------------------------
emoji_keywords = {
    'coreana': '🥢',
    'japonesa': '🍣',
    'peruana': '🥑',
    'española': '🥘',
    'francesa': '🥐',
    'india': '🍛',
    'tailandesa': '🍲',
    'chilena': '🌽',
    'norteamericana': '🍔',
    'latinoamericana': '🌮',
    'mediterránea': '🥗',
    'italiana': '🍕',
    'asiática': '🍜',
    'otra': '🍴'
}

def asignar_varios_emojis(origen_texto):
    if pd.isna(origen_texto):
        return '🍴'
    origen_lower = origen_texto.lower()
    emojis = []
    for palabra, emoji in emoji_keywords.items():
        if palabra in origen_lower:
            emojis.append(emoji)
    return ' '.join(emojis) if emojis else '🍴'

df['emoji'] = df['Origen geográfico'].apply(asignar_varios_emojis)

# -------------------------
# 4️⃣ Crear columna de posición horizontal dentro de cada rating
# -------------------------
df['x_pos'] = df.groupby('Rating').cumcount()

# -------------------------
# 5️⃣ Crear gráfico tipo pictograma apilado (rotado)
# -------------------------
chart = alt.Chart(df).mark_text(fontSize=20).encode(
    y=alt.Y('Rating:Q', title='Rating', scale=alt.Scale(domain=[0,5])),
    x=alt.X('x_pos:O', title=None, axis=None),
    column=alt.Column('Origen geográfico:N', title='Origen geográfico',
                      header=alt.Header(labelFontSize=12, labelAngle=-45, labelPadding=5)),
    text='emoji:N',
    tooltip=[
        alt.Tooltip('Nombre:N', title='Restaurante'),
        alt.Tooltip('Origen geográfico:N', title='Origen'),
        alt.Tooltip('Rating:Q', title='Rating', format=".1f"),
        alt.Tooltip('ranking_num:Q', title='Ranking', format=".1f")
    ]
).properties(
    width=300,    # ancho de cada columna
    height=300   # alto del eje vertical
).configure_view(
    strokeWidth=0
).configure_header(
    labelFontSize=12,
    labelAngle=-40
)

chart


alt.Chart(...)

In [36]:
import pandas as pd
import altair as alt
import re
import numpy as np

# -------------------------
# 1️⃣ Cargar datos
# -------------------------
df = pd.read_csv("/content/base_tripadvisor_visualización_cocina_asiática.csv", sep=';', on_bad_lines='skip')
df.columns = df.columns.str.strip()
df['Rating'] = pd.to_numeric(df['Rating (1-5)'], errors='coerce')

# -------------------------
# 2️⃣ Agrupar promedios por origen geográfico
# -------------------------
ratings = df.groupby('Origen geográfico', as_index=False)['Rating'].mean()

# -------------------------
# 3️⃣ Umbral de destacación (por ejemplo, 4.5)
# -------------------------
threshold_value = 4.5
threshold = pd.DataFrame([{"threshold": threshold_value}])

# -------------------------
# 4️⃣ Gráfico de barras con segmento destacado
# -------------------------

# Barras base
bars = alt.Chart(ratings).mark_bar(color="#a3c4f3").encode(
    x=alt.X('Origen geográfico:N', title='Origen geográfico', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('Rating:Q', title='Rating promedio')
)

# Segmento destacado
highlight = alt.Chart(ratings).mark_bar(color="#e45755").encode(
    x='Origen geográfico:N',
    y=alt.Y('baseline:Q'),
    y2='Rating:Q'
).transform_filter(
    alt.datum.Rating > threshold_value
).transform_calculate(
    baseline=str(threshold_value)
)

# Línea del umbral
rule = alt.Chart(threshold).mark_rule(color='red', strokeDash=[4, 2]).encode(
    y='threshold:Q'
)

# Combinar todo
chart = (bars + highlight + rule).properties(
    width=600,
    height=400,
    title=f"Rating promedio por origen geográfico (destacando > {threshold_value})"
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_title(
    fontSize=14,
    anchor='start'
)

chart



alt.LayerChart(...)